### 지도학습 기반 감성 분석 실습 – Kaggle_IMDB 영화평

In [ ]:
#출처: 파이썬 머신러닝 완벽 가이드 , 위키북스, p497

## 데이터 수집 및 이해

In [ ]:
# 데이터셋
print("https://www.kaggle.com/c/word2vec-nlp-tutorial/data")

In [1]:
import pandas as pd
review_df = pd.read_csv('../data/labeledTrainData.tsv', header=0, sep='\t', quoting=3)

In [2]:
review_df

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."
...,...,...,...
24995,"""3453_3""",0,"""It seems like more consideration has gone int..."
24996,"""5064_1""",0,"""I don't believe they made this film. Complete..."
24997,"""10905_3""",0,"""Guy is a loser. Can't get girls, needs to bui..."
24998,"""10194_3""",0,"""This 30 minute documentary Buñuel made in the..."


In [4]:
review_df.loc[0, 'review']

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

## 데이터 준비

In [5]:
import re

# <br> html 태그는 replace 함수로 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />',' ')

# 파이썬의 정규 표현식 모듈인 re를 이용하여 영어 문자열이 아닌 문자는 모두 공백으로 변환
review_df['review'] = review_df['review'].apply(lambda x: re.sub("[^a-zA-Z]", " ", x))

In [6]:
# 타켓 데이터 세트와 피처 데이터 세트 생성
class_df = review_df['sentiment']
feature_df = review_df.drop(['id', 'sentiment'], axis=1, inplace=False)

In [7]:
# 학습용 / 테크트용 데이터 세트 분할
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

X_train.shape, X_test.shape

((17500, 1), (7500, 1))

## 데이터 모델링

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))
])

In [12]:
pipeline.fit(X_train['review'], y_train)

/Users/kimminsoo/miniconda/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('cnt_vect',
                 CountVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('lr_clf', LogisticRegression(C=10))])

In [13]:
y_pred = pipeline.predict(X_test['review'])

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[3250  430]
 [ 425 3395]]
0.886


# [실습] Starbucks review Data

In [20]:
star_df = pd.read_csv('../data/Starbucks_review_data/train.csv')
star_df.head(5)

,label,document
0,0,주말에 사람이 적은 편. 2층 와이파이가 안돼서 1층 와이파이에 연결했는데 불안정함.
1,1,파트너분들 진짜 세상 친절하시고 너무 좋아요ㅎ
2,1,방금 확인했어요~
3,0,별적립 12개 무료음료쿠폰으로는 프리퀀시적립이 안된다기에 (작년에는 됐던것같아서) ...
4,1,요 근처에서 보기드물게 넓고 쾌적한 스벅이네요~


In [21]:
star_df.loc[4, 'document']

'요 근처에서 보기드물게 넓고 쾌적한 스벅이네요~'

In [22]:
import re

star_df['document'] = star_df['document'].str.replace('<br /', ' ')

In [23]:
class_df = star_df['label']
feature_df = star_df.drop(['label'], axis=1, inplace=False)

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

((3037, 1), (760, 1))

In [25]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))
])

In [26]:
pipeline.fit(X_train['document'], y_train)

Pipeline(steps=[('cnt_vect', CountVectorizer(ngram_range=(1, 2))),
                ('lr_clf', LogisticRegression(C=10))])

In [27]:
y_pred = pipeline.predict(X_test['document'])

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[126  86]
 [ 16 532]]
0.8657894736842106


In [29]:
train_df = pd.read_csv('../data/Starbucks_review_data/test.csv', encoding='cp949')
train_df

,no,document
0,1,화장실이 좀 별로지만 직원분들 다 친절하시고 매장 깔끔함
1,2,최고예요
2,3,서비스 굿
3,4,"친절하고, 자리 넓고 좋아요"
4,5,최고예요
...,...,...
1261,1262,드라이브쓰루 좋아요
1262,1263,사이렌오더 확인안함.. 직원 5명 아무것도 안하고 수다떨고.. 매장와서 사이렌오더...
1263,1264,스타벅스는 직원교육으로 유명한데.. 여기 스탭님들은 왜....
1264,1265,조금 아쉬워요


In [35]:
#q_test = (train_df['document'])
q_pred = pipeline.predict(train_df['document'])

ValueError: X has 13966 features, but LogisticRegression is expecting 26872 features as input.